In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

from scipy import stats
from scipy.stats import norm, skew #for some statistics

pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))

from subprocess import check_output

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
train = pd.read_csv('/content/drive/My Drive/データ解析コンペ/Signate/ひろしまQuest 野球コンペ/球速部門/train_pitch.csv')
test = pd.read_csv('/content/drive/My Drive/データ解析コンペ/Signate/ひろしまQuest 野球コンペ/球速部門/test_pitch.csv')
sub = pd.read_csv("/content/drive/My Drive/データ解析コンペ/Signate/ひろしまQuest 野球コンペ/球速部門/sample_submit_ball_type.csv")

In [ ]:
train_ID = train['データ内連番']
test_ID = test['データ内連番']

train_BallType = train['球種']

In [ ]:
train.drop("データ内連番", axis = 1, inplace = True)
test.drop("データ内連番", axis = 1, inplace = True)

In [ ]:
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.球種.values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['球種'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

all_data size is : (778767, 49)


In [ ]:
all_data.drop(['投球位置区域','試合種別詳細', '表裏','日付','時刻', '投手試合内対戦打者数', '打者チームID', '打者試合内打席数',
           'ホームチームID', 'アウェイチームID', 'イニング内打席数', '試合内連番', '球場名', '投手登板順', '一塁走者ID', '二塁走者ID', '三塁走者ID'], axis = 1, inplace = True)

In [ ]:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head()


Skew in numerical features: 



,Skew
投手チームID,3.003
球場ID,2.925
プレイ前ホームチーム得点数,1.611
プレイ前アウェイチーム得点数,1.475
投手イニング内投球数,1.005


In [ ]:
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    all_data[feat] = boxcox1p(all_data[feat], lam)

There are 27 skewed numerical features to Box Cox transform


In [ ]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)

(778767, 53)


In [ ]:
len(test)

521650

In [ ]:
train2 = all_data[:len(train)]
test2 = all_data[len(train):]
train_2ID = all_data[:len(train)]
test_2ID = all_data[len(train):]

In [ ]:
train = pd.concat([train_2ID, train_BallType, train2], axis = 1)
test = pd.concat([test_2ID, test2], axis = 1)

In [ ]:
y_train = train['球種']
X_train = train.drop('球種', axis = 1)
X_test = test

In [ ]:
test.shape

(521650, 106)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

In [ ]:
pip install optuna

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.33)

In [ ]:
test.shape

(521650, 106)

In [ ]:
print(X_train.shape)
print(X_valid.shape)

(172268, 106)
(84849, 106)


In [ ]:
categorical_features = list(train.columns)

In [ ]:
categorical_features.remove('球種')

**Model: LightGBM**

In [ ]:
import lightgbm as lgb
import optuna
from sklearn.metrics import log_loss

def objective(trial):
    params = {
        'objective':'multiclass', 
        'max_bin': trial.suggest_int('max_bin', 255, 500), 
        'learning_rate':0.05, 
        'num_leaves': trial.suggest_int('num_leaves', 32, 128),
        'num_class': 8
    }
    
    lgb_train = lgb.Dataset(X_train, y_train, 
                           categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference = lgb_train,  
                          categorical_feature=categorical_features)
    
    model = lgb.train(params, lgb_train, 
                     valid_sets = [lgb_train, lgb_eval], 
                     verbose_eval = 100, 
                     num_boost_round = 1000, 
                     early_stopping_rounds = 10)
    
    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)
    score = log_loss(y_valid, y_pred_valid)
    return score

In [ ]:
study = optuna.create_study(sampler = optuna.samplers.RandomSampler(seed = 0))
study.optimize(objective, n_trials = 40)
study.best_params

In [ ]:
import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, y_train, 
                       categorical_feature = categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference = lgb_train, 
                      categorical_feature=categorical_features)
params = {
    'objective': 'multiclass', 
    'metric': {'multi_logloss'},
    'num_class': 8, 
    'max_bin': 332, 
    'num_leaves': 104
}

model = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    num_boost_round = 1000,
    verbose_eval=50,
    early_stopping_rounds = 10)

y_pred = model.predict(X_test, num_iteration = model.best_iteration)

Training until validation scores don't improve for 10 rounds.
[50]	training's multi_logloss: 1.13905	valid_1's multi_logloss: 1.21851
[100]	training's multi_logloss: 1.031	valid_1's multi_logloss: 1.18265
[150]	training's multi_logloss: 0.962221	valid_1's multi_logloss: 1.1743
[200]	training's multi_logloss: 0.906998	valid_1's multi_logloss: 1.1713
Early stopping, best iteration is:
[233]	training's multi_logloss: 0.87493	valid_1's multi_logloss: 1.1711


In [ ]:
y_pred.shape

(521650, 8)

In [ ]:
sub = pd.DataFrame(y_pred,columns=['0.125', '0.125', '0.125', '0.125', '0.125', '0.125','0.125', '0.125']).reset_index()
sub.columns = ['0', '0.125', '0.125', '0.125', '0.125', '0.125', '0.125','0.125', '0.125']
sub.to_csv('sub_lightgbm.csv', index=False)

In [ ]:
from google.colab import files
files.download('sub_lightgbm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Cross Validation**

In [ ]:
t4_params = {
    'boosting_type': 'gbdt', 'objective': 'multiclass', 'nthread': -1, 'silent': True,
    'num_leaves': 2**4, 'learning_rate': 0.05, 'max_depth': -1,
    'max_bin': 255, 'subsample_for_bin': 50000,
    'subsample': 0.8, 'subsample_freq': 1, 'colsample_bytree': 0.6, 'reg_alpha': 1, 'reg_lambda': 0,
    'min_split_gain': 0.5, 'min_child_weight': 1, 'min_child_samples': 10, 'scale_pos_weight': 1}

# they can be used directly to build a LGBMClassifier (which is wrapped in a sklearn fashion)
t4 = lgbm.sklearn.LGBMClassifier(n_estimators=1000, seed=0, **t4_params)

In [ ]:
def cross_validate_lgbm(filename_str, preprocess_func=preprocess_1):
    lgbm_params = t4_params.copy()
    lgbm_params['num_class'] = 3
    train_X, train_y, test_df = preprocess_func(train, test)
    dset = lgbm.Dataset(train_X, train_y, silent=True)
    cv_results = lgbm.cv(
        lgbm_params, dset, num_boost_round=10000, nfold=5, stratified=False, shuffle=True, metrics='multi_logloss',
        early_stopping_rounds=100, verbose_eval=50, show_stdv=True, seed=0)
    # note: cv_results will look like: {"multi_logloss-mean": <a list of historical mean>,
    # "multi_logloss-stdv": <a list of historical standard deviation>}
    print('best n_estimators:', len(cv_results['multi_logloss-mean']))
    print('best cv score:', cv_results['multi_logloss-mean'][-1])

# we simply have to run the following code each time we modify the hyperparameters:
cross_validate_lgbm('lightgbm.csv')


**Cross Validation**

In [ ]:
train.index

RangeIndex(start=0, stop=257117, step=1)

In [ ]:
from sklearn.model_selection import KFold
import lightgbm as lgb

y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)

params = {
    'objective': 'multiclass', 
    'metric': {'multi_logloss'},
    'num_class': 8, 
    'max_bin': 332, 
    'num_leaves': 104
}

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train[:len(train)]
    X_val = X_train[len(train):]
    y_tr = y_train[:len(train)]
    y_val = y_train[len(train):]

    lgb_train = lgb.Dataset(X_tr, y_tr,
                                             categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train,
                                            categorical_feature=categorical_features)

    model = lgb.train(params, lgb_train,
                                   valid_sets=[lgb_train, lgb_eval],
                                   verbose_eval=10,
                                   num_boost_round=1000,
                                   early_stopping_rounds=10)


    oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    y_preds.append(y_pred)
    models.append(model)

ValueError: ignored